In [1]:
from collections import defaultdict
import numpy as np

In [2]:
def read_input(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    return lines


def check_input_format(input_):
    for i in input_:
        if not i.isalnum():
            raise Exception("Node label has to be alphanumeric")
            

def check_missing_leaves(label2val, leaf_nodes, adj_list):
    for leaf in leaf_nodes:
        if leaf not in label2val:
            lis = [li for li in adj_list.values() if leaf in li]
            parent_nodes = [k for li in lis for k in adj_list if adj_list[k]==li]
            raise Exception(f"child node {leaf} of {parent_nodes} not found")


def check_multiple_root(root_set):
    if len(root_set) > 1:
        raise Exception(f"multiple roots: {root_set}")
            

def find_root(child2parent, adj_list):
    root_set = set()
    for node in adj_list:
        if node not in child2parent:
            root_set.add(node)
    check_multiple_root(root_set)
    return next(iter(root_set))            
            
            
def parse_input(filename):
    lines = read_input(filename=filename)
    lines = [l.strip('\n') for l in lines if l != '\n']  #remove extra spaces from input
        
    adj_list = dict()
    label2val = dict()
    child2parent = defaultdict(list)
    labels = set()
    internal_nodes = set()
    leaf_nodes = set()
    
    for line in lines:
        if ":" in line:
            src, children = line.split(": ")[0], line.split(": ")[-1].lstrip('[').rstrip(']').split(", ")
            check_input_format([src])
            check_input_format(children)
            adj_list[src] = children
            for c in children:
                child2parent[c].append(src)
            labels.add(src)
            internal_nodes.add(src)
            labels.update(children)
            label2val[src] = None

        if "=" in line:
            label_, val = line.split("=")[0], int(line.split("=")[-1])
            check_input_format([label_])
            label2val[label_] = val
    
    leaf_nodes = labels - internal_nodes
    check_missing_leaves(label2val=label2val, leaf_nodes=leaf_nodes, adj_list=adj_list)
    root = find_root(child2parent, adj_list)
            
    return adj_list, label2val, root

In [3]:
## Alpha Beta Pruning
## ------------------

class Node_ab:
    def __init__(self, label, value=None, node_type='min', alpha=float("-inf"), beta=float("inf")):
        self.label = label
        self.value = value
        self.node_type = node_type
        self.children = []
        self.alpha = alpha
        self.beta = beta
        
    def __repr__(self,):
        return str(self.label)
    
    def __str__(self,):
        return f"({self.label}, {self.value}, {self.node_type}, {self.alpha}, {self.beta})"
    
    
class DAG_ab:
    def __init__(self, adj_list=None, root_label=None, root_node_type='max', n=None, v=None, label2val=None):
        self.adj_list = adj_list if adj_list else defaultdict(list) # {'a': ['a1', 'a2', 'a3'], ...}
        self.label2node = {}  # {'a': Node('a'), 'a1': Node('a1'), 'a2': Node('a2'), ...}
        self.root_label = root_label # 'a'
        self.root_node_type = root_node_type
        self.root = None
        self.n = n
        self.verbose = v
        self.label2val=label2val
        
    def __repr__(self,):
        return f"DAG({str(self.adj_list)})"
    
    def __str__(self):
        return f"DAG({str(self.adj_list)})"
    
    def create_node_graph(self):
        if self.root_label is None:
            raise ValueError("Root cannot be None before instantiating graph!")
        alpha = float('-inf')
        beta = float('inf')
        self.root = self._create_nodes(self.root_label, self.root_node_type, alpha, beta)
        if not self.verbose:
            choice = [child.label for child in self.root.children if child.value==self.root.value][0]
            print(f"({self.root.node_type}){self.root.label} chooses {choice} for {self.root.value}")

        
    def _create_nodes(self, root_label, root_node_type, root_alpha, root_beta):
        # root_label = 'a'
        
        # Check if root label already has a Node object
        if root_label in self.label2node:
#             print(f"root_label = {root_label} already in label2node.")
            root_node = self.label2node[root_label]
        else:
#             print(f"root_label = {root_label} not in label2node.")
            root_node = Node_ab(label=root_label,
                             value=self.label2val[root_label],
                             node_type=root_node_type,
                             alpha=root_alpha,
                             beta=root_beta)
            self.label2node[root_label] = root_node
         
        
            
        child_nodes_visited = []
        local_verbose = True

        if root_label in self.adj_list:
#             print(f"Present: {self.adj_list[root_label]}")
#             print(f"Rootnode: {root_node.label}, alpha:{root_node.alpha}, beta:{root_node.beta}")
            for child_label in self.adj_list[root_label]:
                if condition(root_node.alpha, root_node.beta):
#                     print(f"pruning child: {child_node.label} of parent: {root_label}.........")
                    local_verbose = False
                    break;
                if root_node_type == 'max':
                    child_node_type = 'min'
                if root_node_type == 'min':
                    child_node_type = 'max'
                child_node = self._create_nodes(child_label, child_node_type, root_node.alpha, root_node.beta)
                if root_node.node_type == 'max':
                    if child_node.value > root_node.alpha:
                        root_node.alpha = child_node.value
                        root_node.value = root_node.alpha
                        choice = child_node.label
                        if self.n and root_node.value == self.n:
                            if self.verbose and local_verbose:
                                print(f"({root_node_type}){root_label} chooses {choice} for {root_node.value}")
                            return root_node
                if root_node.node_type == 'min':
                    if child_node.value < root_node.beta:
                        root_node.beta = child_node.value
                        root_node.value = root_node.beta
                        choice = child_node.label
                        if self.n and root_node.value == -self.n:
                            if self.verbose and local_verbose:
                                print(f"({root_node_type}){root_label} chooses {choice} for {root_node.value}")
                            return root_node

                root_node.children.append(child_node)
            if local_verbose and verbose:
                if not condition(root_node.alpha, root_node.beta):
                    print(f"({root_node_type}){root_label} chooses {choice} for {root_node.value}")
    
        return root_node

In [9]:
adj_list, label2val, root = parse_input("example3.txt")
verbose=True
dag_ab = DAG_ab(adj_list=adj_list, root_label=root, root_node_type='max', label2val=label2val, v=verbose)
dag_ab.create_node_graph()

(max)d chooses h for 10
(min)b chooses d for 10
(max)a chooses b for 10


In [5]:
def condition(alpha, beta):
    return beta <= alpha